# Creating a convolutional network from scratch

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers

conv_network = models.Sequential()
conv_network.add(layers.Conv2D(20, (5,5), activation='relu', input_shape=(232, 154, 3)))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.Conv2D(20, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.MaxPooling2D(3,3))
conv_network.add(layers.Conv2D(20, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.Conv2D(10, (5,5), activation='relu'))
conv_network.add(layers.Dropout(0.2))
conv_network.add(layers.MaxPooling2D(3,3))
conv_network.add(layers.Flatten())
conv_network.add(layers.Dense(4, activation='softmax'))

optimizer=tf.keras.optimizers.Adam(learning_rate=0.02)

conv_network.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

conv_network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 228, 150, 20)      1520      
                                                                 
 dropout (Dropout)           (None, 228, 150, 20)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 146, 20)      10020     
                                                                 
 dropout_1 (Dropout)         (None, 224, 146, 20)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 48, 20)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 70, 44, 20)        10020     
                                                        

### Loading in the data

In [20]:
data_dir = "/Users/kerickwalker/src/dis/deep_learning/bat_data"

img_width = 154
img_height = 232
batch_size = 128

# Load in the training data
training_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Load in validation data
validation_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2008 files belonging to 4 classes.
Using 1607 files for training.
Found 2008 files belonging to 4 classes.
Using 401 files for validation.


### Training our network

In [ ]:
conv_network.fit(training_data, validation_data=validation_data, epochs=10)

# Transfer Learning with MobileNetV2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

2023-12-04 12:33:22.672010: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Note that the file paths here refer to my local file structure:
# I have a training data directory and a test data directory
# Both directories contain sub_directories with corresponding names
# The test data directory contains 10% of the total data.
# We are employing an 80/10/10 train, validation, test split

img_size = (232, 154)
batch_size = 32
train_data_dir = "/Users/kerickwalker/src/dis/deep_learning/train_bat_data"
test_data_dir = "/Users/kerickwalker/src/dis/deep_learning/test_bat_data"

In [6]:
# Generate training and validation data

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.11
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

Found 1612 images belonging to 4 classes.
Found 197 images belonging to 4 classes.


In [7]:
# Generate test data

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

Found 199 images belonging to 4 classes.


In [8]:
base_model = MobileNetV2(
    input_shape=(232, 154, 3),
    include_top=False,
    weights='imagenet'
)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [16]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)

Epoch 1/10


2023-12-04 12:47:37.966974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 46s 865ms/step - loss: 0.1477 - accuracy: 0.9424 - val_loss: 0.3632 - val_accuracy: 0.8934
Epoch 2/10
50/50 [==============================] - 44s 886ms/step - loss: 0.1585 - accuracy: 0.9392 - val_loss: 0.2646 - val_accuracy: 0.9289
Epoch 3/10
50/50 [==============================] - 47s 928ms/step - loss: 0.1477 - accuracy: 0.9449 - val_loss: 0.3426 - val_accuracy: 0.9036
Epoch 4/10
50/50 [==============================] - 45s 890ms/step - loss: 0.1364 - accuracy: 0.9405 - val_loss: 0.3844 - val_accuracy: 0.9137
Epoch 5/10
50/50 [==============================] - 45s 888ms/step - loss: 0.1287 - accuracy: 0.9475 - val_loss: 0.3218 - val_accuracy: 0.9036
Epoch 6/10
50/50 [==============================] - 45s 891ms/step - loss: 0.1275 - accuracy: 0.9500 - val_loss: 0.4608 - val_accuracy: 0.9036
Epoch 7/10
50/50 [==============================] - 43s 863ms/step - loss: 0.1350 - accuracy: 0.9456 - val_loss: 0.2310 - val_accuracy: 0.9289
Epoch 8/10

In [18]:
score = model.evaluate(
    x = test_generator, 
    batch_size = batch_size
)

2023-12-04 12:55:08.377453: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 5s 694ms/step - loss: 0.1528 - accuracy: 0.9447


In [19]:
# Saves the model to the github repository
# (Again, file path refers to my local file structure)

model.save('/Users/kerickwalker/src/dis/deep_learning/bat/model.keras')